In [1]:
import sys
import os

# Absolute path to your project root
project_root = "/Users/merlijnbroekers/Desktop/Drone_Interception"

# Add it to Python path if not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Current working dir :", os.getcwd())
print("Project root        :", project_root)
print("src/ exists?        :", os.path.exists(os.path.join(project_root, "src")))


Current working dir : /Users/merlijnbroekers/Desktop/Drone_Interception/scripts/analysis
Project root        : /Users/merlijnbroekers/Desktop/Drone_Interception
src/ exists?        : True


In [2]:
from dataclasses import dataclass
from pathlib import Path
import copy, glob, pickle
import numpy as np

from src.models.evaders.registry import build_evader
from src.models.pursuers.registry import build_pursuer
from scripts.analysis.utils.metrics_utils import compute_simulation_metrics

In [3]:
CONFIG = {
    "DT": 1 / 100,
    "TOTAL_TIME": 10,
    "TIME_LIMIT": 1000,
    "STOP_ON_INTERCEPTION": False,
    "INTERCEPTION_RADIUS": 0.15,
    "CAPTURE_RADIUS": 0.15,
    "CAPTURE_PENALTY": -10,
    "ENV_BOUND": 10,
    "OUT_OF_BOUNDS_PENALTY": 5,
    "RATE_PENALTY": 0.001,
    "OBSERVATIONS": {
        "OBS_MODE": "pos+vel",
        "MAX_HISTORY_STEPS": 1,
        "INCLUDE_HISTORY": False,
        "HISTORY_STEPS": 0,
        "INCLUDE_ACTION_HISTORY": False,
        "ACTION_HISTORY_STEPS": 0,
        "ACTION_DIM": 4,
        "OPTIONAL_FEATURES": [
            # "attitude",
            "attitude_mat",
            "rates",
            # "T_force",
            "omega_norm",
        ],
    },
    "reward_type": "effective_gain",
    "SMOOTHING_GAMMA": 0.4,
    "MOTH_FOLDER": "opogona_moth_data/top_moths",
    "PURSUER": {
        "MODEL": "motor",
        "LOG_LEVEL": "DEBUG",
        "INITIAL_POS": [0.0, 0.0, 0.0],
        "INIT_RADIUS": 1.0,
        "INITIAL_VEL": [0.0, 0.0, 0.0],
        "INITIAL_ATTITUDE": [0.0, 0.0, 0.0],
        "INITIAL_RATES": [0.0, 0.0, 0.0],
        "INITIAL_OMEGA": [0.0, 0.0, 0.0],
        "MAX_ACCELERATION": 20,
        "MAX_SPEED": 20,
        "ACTUATOR_TAU": 0.02,
        "POSITION_NOISE_STD": 0.00,
        "VELOCITY_NOISE_STD": 0.00,
        "BUTTER_ACC_FILTER_CUTOFF_HZ": 8,
        "gravity": 9.81,
        "face_evader": False,
        "mass": 1.0,
                "BOUNDARIES": {
            # fall back boundaries if no planes are defined
            "ENV_BOUNDS": {"x": (-10.0, 10.0), "y": (-10.0, 10.0), "z": (-10.0, 10)},
            "BOUNDARY_MARGIN": 0.1,  # distance from wall where penalty starts
            "BOUNDARY_PENALTY_WEIGHT": 0.25,  # global multiplier
            "BOUNDARY_MODE": "sum",  # "sum" (default) or "max" (no corner boost)
            "PLANES": None,
            # [  # Set to none to fallback to ENV_bounds
            #     {
            #         "n": [0.5961325493620492, 0.7253743710122877, 0.3441772878468769],
            #         "p0": [0.0, 0.0, 0.0],
            #     },
            #     {
            #         "n": [0.5961325493620492, -0.7253743710122877, 0.3441772878468769],
            #         "p0": [0.0, 0.0, 0.0],
            #     },
            #     {
            #         "n": [0.8571673007021123, 0.0, -0.5150380749100543],
            #         "p0": [0.0, 0.0, 0.0],
            #     },
            #     {
            #         "n": [-0.017452406437283352, 0.0, 0.9998476951563913],
            #         "p0": [0.0, 0.0, 0.0],
            #     },
            #     {
            #         "n": [0.8660254037844387, 0.0, 0.49999999999999994],
            #         "p0": [0.47631397208144133, 0.0, 0.27499999999999997],
            #     },
            #     {
            #         "n": [-0.8660254037844387, -0.0, -0.49999999999999994],
            #         "p0": [3.464101615137755, 0.0, 1.9999999999999998],
            #     },
            #     {"n": [0.0, 0.0, -1.0], "p0": [0.0, 0.0, 2.0]},
            #     {"n": [0.0, 0.0, 1.0], "p0": [0.0, 0.0, -0.1]},
            #     {"n": [-1.0, 0.0, 0.0], "p0": [3.0, 0.0, 0.0]},
            #     {"n": [1.0, 0.0, 0.0], "p0": [0.55, 0.0, 0.0]},
            # ],
        },
        "actuator_time_constants": {
            "p": 1.646387e-01,
            "q": 1.515623e-01,
            "r": 4.450389e-01,
            "T": 7.771194e-02,
            "phi": 0.05,
            "theta": 0.05,
        },
        "actuator_limits": {
            "p": (-3.0, 3.0),
            "q": (-3.0, 3.0),
            "r": (-2.0, 2.0),
            "T": (0.0, 18.0),
            "bank_angle": 20.0,
        },
        "attitude_pd_gains": {
            "kp": np.array([500.0, 500.0, 500.0]),
            "kd": np.array([1, 1, 1]),
        },
        "delta_a_limits": {
            "min": [-6.0, -6.0, -9.0],
            "max": [6.0, 6.0, 9.0],
        },
        "motor": {
            "k_w": -3.697721e-06,
            "k_x": -1.019900e-04,
            "k_y": -1.051760e-04,
            "k_p1": 9.165652e-05,
            "k_p2": -7.968586e-05,
            "k_p3": -9.095348e-05,
            "k_p4": 8.340457e-05,
            "k_q1": 8.364455e-05,
            "k_q2": 7.527732e-05,
            "k_q3": -8.162811e-05,
            "k_q4": -7.600954e-05,
            "k_r1": 6.126003e-03,
            "k_r2": -1.026074e-02,
            "k_r3": 9.441550e-03,
            "k_r4": -6.165215e-03,
            "k_r5": 2.115877e-03,
            "k_r6": -1.801040e-03,
            "k_r7": 1.627792e-03,
            "k_r8": -1.797608e-03,
            "w_min": 3.089704e02,
            "w_max": 1.174345e03,
            "curve_k": 1.000000e00,
            "tau": 4.791173e-02,
        },
        "drag": {"kx_acc_ctbr": 3.107232e-01, "ky_acc_ctbr": 3.280459e-01},
        "thrust_limits": {"T_min": 0.0, "T_max": 18.0},
        "domain_randomization_pct": {
            "g": 0.00,
            "kx_acc_ctbr": 0.0,
            "ky_acc_ctbr": 0.0,
            "taup": 0.0,
            "tauq": 0.0,
            "taur": 0.0,
            "tauT": 0.0,
            "max_accel": 0.0,
            "max_speed": 0.0,
            "p_lo": 0.0,
            "p_hi": 0.0,
            "q_lo": 0.0,
            "q_hi": 0.0,
            "r_lo": 0.0,
            "r_hi": 0.0,
            "T_lo": 0.0,
            "T_hi": 0.0,
            "bank_angle": 0.0,
            "max_accel": 0.0,
            "delta_a_min": 0.0,
            "delta_a_max": 0.0,
            "init_radius": 0.0,
            "k_x": 0.0,
            "k_y": 0.0,
            "k_w": 0.0,
            "k_p1": 0.0,
            "k_p2": 0.0,
            "k_p3": 0.0,
            "k_p4": 0.0,
            "k_q1": 0.0,
            "k_q2": 0.0,
            "k_q3": 0.0,
            "k_q4": 0.0,
            "k_r1": 0.0,
            "k_r2": 0.0,
            "k_r3": 0.0,
            "k_r4": 0.0,
            "k_rd1": 0.0,
            "k_rd2": 0.0,
            "k_rd3": 0.0,
            "k_rd4": 0.0,
            "w_min": 0.0,
            "w_max": 0.0,
            "tau": 0.0,
            "curve_k": 0.0,
            "init_radius": 0.0,
        },
        "CONTROLLER": {
            "type": "rl",  #   "rl" | "frpn" | "lpn" | …
            "policy_path": "/Users/merlijnbroekers/Desktop/Drone_Interception/observation_testing/models/pos+vel/best_model.zip",  #   only for RL
            "params": {  #   whatever the ctor of the class needs
                "lambda_": 180,
                "pp_weight": 0.25,
                "max_acceleration": 10,
            },
        },
    },
    "EVADER": {
        "MODEL": "moth",  # "classic", "moth", "rl", "pliska"
        "CSV_FILE": "/Users/merlijnbroekers/Desktop/Drone_Interception/top_5_csv/log_itrk17.csv",
        "PATH_TYPE": "figure_eight",
        "RADIUS": 0.0,
        "VELOCITY_MAGNITUDE": 3.0,
        "NOISE_STD": 0.00,
        "NOISE_STD_POS": 0.00,
        "NOISE_STD_VEL": 0.00,
        "FILTER_TYPE": "passthrough",
        "FILTER_PARAMS": {
            "process_noise": 1e-4,
            "measurement_noise": 1e-2,
        },
        "RL_MODEL_PATH": "final_models/Evader_a10_v10_0613_1732/ppo_checkpoint_step_20000000.zip",
        "MAX_ACCEL": 20,
        "MAX_SPEED": 2,
        "INIT_POS": [0, 0, 0],
        "INIT_VEL": [0, 0, 0],
    },
}

In [4]:
from scripts.analysis.utils.notebook_helpers import AnalysisRunConfig

RUN_CFG = AnalysisRunConfig(
    n_trials=1,
    evader_csv_dir="/Users/merlijnbroekers/Desktop/Drone_Interception/evader_datasets/top_10_csv",
    out_dir=str(Path(project_root) / "analysis_out/trials"),
)

# Toggle recompute/save vs load
RUN_TOGGLE = {
    "RECOMPUTE": False,   # set True to run simulations
    "SAVE_RESULTS": False # set True to save to pickle after running
}


In [5]:
def discover_policies():
    return {Path(p).parent.name: p for p in glob.glob("/Users/merlijnbroekers/Desktop/Drone_Interception/trained_models/observation_testing/models/*/best_model.zip")}


def build_motor_pursuer(policy_path: str, cfg: dict):
    pcfg = copy.deepcopy(cfg["PURSUER"])
    pcfg["CONTROLLER"] = copy.deepcopy(pcfg["CONTROLLER"])
    pcfg["CONTROLLER"]["policy_path"] = str(Path(policy_path).with_suffix(".zip"))
    return build_pursuer(pcfg, cfg)

In [6]:
from scripts.analysis.utils.run_utils import Variant, run_sweep_variants
from scripts.analysis.utils.run_utils import list_evaders 

def run_all_simulations_by_obs_mode():
    evaders = list(list_evaders(RUN_CFG.evader_csv_dir, extra_fields={"NOISE_STD": CONFIG["EVADER"]["NOISE_STD"]}))
    policies = discover_policies()
    variants = [Variant(key=k, meta={"policy_path": v}) for k, v in policies.items()]

    def build_cfg_fn(ev_cfg, obs_mode, meta):
        cfg = copy.deepcopy(CONFIG)
        cfg["OBSERVATIONS"]["OBS_MODE"] = obs_mode
        cfg["EVADER"].update({k: v for k, v in ev_cfg.items() if k != "label"})
        return cfg

    def build_agents_fn(cfg, obs_mode, meta):
        evader = build_evader(cfg["EVADER"], cfg)
        pursuer = build_motor_pursuer(meta["policy_path"], cfg)
        return pursuer, evader

    def extra_fields_fn(ev_cfg, obs_mode, meta, metrics, sim):
        return {"obs_mode": obs_mode}

    df = run_sweep_variants(
        evaders=evaders,
        variants=variants,
        n_trials=RUN_CFG.n_trials,
        build_cfg_fn=build_cfg_fn,
        build_agents_fn=build_agents_fn,
        compute_metrics_fn=compute_simulation_metrics,
        extra_fields_fn=extra_fields_fn,
        per_evader_seed_bank=False,
    )
    return df


In [7]:
trials_path = Path(project_root) / "analysis_out/trials/obs_space.pkl"

In [8]:
# Compute or load trials based on toggles
if RUN_TOGGLE.get("RECOMPUTE", False):
    trials = run_all_simulations_by_obs_mode()
    if RUN_TOGGLE.get("SAVE_RESULTS", False):
        with open(trials_path, "wb") as f:
            pickle.dump(trials, f)
else:
    with open(trials_path, "rb") as f:
        trials = pickle.load(f)

In [9]:
from scripts.analysis.utils.metrics_utils import aggregate_metrics_grouped

aggregate_metrics_grouped(trials, group_cols=["obs_mode"], success_only_cols={"first_interception_time", "num_clusters", "total_interceptions"})


Group: all
  first_interception_time   mean=2.791 std=1.641 med=2.380 Q1=1.640 Q3=3.297
  closest_distance          mean=0.061 std=0.039 med=0.055 Q1=0.027 Q3=0.074
  time_in_near_miss         mean=19.461 std=8.576 med=21.628 Q1=14.311 Q3=23.826
  total_interceptions       mean=64.100 std=35.430 med=67.500 Q1=38.750 Q3=89.250
  num_clusters              mean=2.400 std=1.200 med=2.500 Q1=1.250 Q3=3.000
  mean_distance             mean=1.669 std=2.130 med=1.057 Q1=0.655 Q3=1.355
  std_distance              mean=1.334 std=1.334 med=0.937 Q1=0.362 Q3=1.654
  interception_flag         mean=1.000 std=0.000 med=1.000 Q1=1.000 Q3=1.000
  ------------------------------------------------------------

Group: all_body_no_phi_rate
  first_interception_time   mean=1.511 std=0.529 med=1.350 Q1=1.143 Q3=1.773
  closest_distance          mean=0.029 std=0.013 med=0.030 Q1=0.017 Q3=0.036
  time_in_near_miss         mean=60.809 std=10.821 med=60.989 Q1=52.123 Q3=66.733
  total_interceptions       mean=18

In [10]:
obs_mode_symbols = {
    "all": r"$x, v, r, \dot{r}, \dot{\phi}$",
    "all_no_phi_rate": r"$x, v, r, \dot{r}$",
    "all_body_no_phi_rate": r"$x, v, r_b, \dot{r}_b$",
    "pos+vel": r"$x, v$",
    "rel_pos+vel": r"$r, \dot{r}$",
    "rel_pos": r"$r$",
    "rel_pos_body": r"$r_b$",
    "rel_pos+vel_body": r"$r_b, \dot{r}_b$",
    "rel_pos_vel_los_rate": r"$r, \dot{r}, \dot{\phi}$",
    "rel_pos_vel_los_rate_body": r"$r_b, \dot{r}_b, \dot{\phi}$",
}

trials["obs_mode_label"] = trials["obs_mode"].map(obs_mode_symbols)


In [11]:
from pathlib import Path
from scripts.analysis.utils.plot_utils import save_obs_boxplot, save_obs_barplot

# Settings
INCLUDE_OBS_MODES = list(obs_mode_symbols.keys())  # or manually: ["pos+vel", "rel_pos+vel_body", ...]
OUTPUT_DIR = Path(project_root) / "figures/observation_modes"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

INCLUDE_OBS_MODES = [
    # "all"
    "all_no_phi_rate",
    # "pos+vel",
    "rel_pos+vel",
    "all_body_no_phi_rate",
    # "rel_pos",
    # "rel_pos_body",
    "rel_pos+vel_body",
    # "rel_pos_vel_los_rate",
    # "rel_pos_vel_los_rate_body",
]

# Filter
df = trials.copy()
df = df[df["obs_mode"].isin(INCLUDE_OBS_MODES)].copy()
df["obs_mode_label"] = df["obs_mode"].map(obs_mode_symbols)
intercepted = df[df["interception_flag"] == 1].copy()

# Save all figures
save_obs_boxplot(intercepted, "first_interception_time", "First Interception Time by Observation Mode", "Time (s)", "first_interception_time.png", OUTPUT_DIR)
save_obs_boxplot(intercepted, "closest_distance", "Closest Distance by Observation Mode", "Distance (m)", "closest_distance.png", OUTPUT_DIR)
save_obs_boxplot(intercepted, "time_in_near_miss", "Time in Near Miss by Observation Mode", "Time (s)", "time_in_near_miss.png", OUTPUT_DIR)
save_obs_boxplot(intercepted, "num_clusters", "Number of Interception Clusters by Observation Mode", "Clusters", "num_clusters.png", OUTPUT_DIR)
save_obs_barplot(df, "Interception Rate by Observation Mode", "Interception Rate", "interception_rate.png", OUTPUT_DIR)

print(f"All observation mode plots saved to: {OUTPUT_DIR.resolve()}")


All observation mode plots saved to: /Users/merlijnbroekers/Desktop/Drone_Interception/figures/observation_modes
